# Heat conduction in a two-partitioned rectangular domain

https://github.com/precice/tutorials/tree/master/partitioned-heat-conduction

## Summary
We solve a simple heat equation. The domain is partitioned and the coupling is established in a Dirichlet-Neumann fashion.

## Setup

We solve a partitioned heat equation. For information on the non-partitioned case, please refer to [1, p.37ff]. In this tutorial the computational domain is partitioned and coupled via preCICE. The coupling roughly follows the approach described in [2].

![Case setup of partitioned-heat-conduction case](images/tutorials-partitioned-heat-conduction-setup.png)

Case setup from [3]. `D` denotes the Dirichlet participant and `N` denotes the Neumann participant.

The heat equation is solved on a rectangular domain `Omega = [0,2] x [0,1]` with given Dirichlet boundary conditions. We split the domain at `x_c = 1` using a straight vertical line, the coupling interface. The left part of the domain will be referred to as the Dirichlet partition and the right part as the Neumann partition. To couple the two participants we use Dirichlet-Neumann coupling. Here, the Dirichlet participant receives Dirichlet boundary conditions (`Temperature`) at the coupling interface and solves the heat equation using these boundary conditions on the left part of the domain. Then the Dirichlet participant computes the resulting heat flux (`Flux`) from the solution and sends it to the Neumann participant. The Neumann participant uses the flux as a Neumann boundary condition to solve the heat equation on the right part of the domain. We then extract the temperature from the solution and send it back to the Dirichlet participant. This establishes the coupling between the two participants.

This simple case allows us to compare the solution for the partitioned case to a known analytical solution (method of manufactures solutions, see [1, p.37ff]). For more usage examples and details, please refer to [3, sect. 4.1].

In [1]:
from pyiron_continuum import Project

In [2]:
pr = Project('partitioned_heat_conduction')
pr.remove_jobs_silently(recursive=True)

/tmp/ipykernel_20028/2810037951.py:2: DeprecationWarning: pyiron_base.project.generic.remove_jobs_silently is deprecated: Use pr.remove_jobs(silently=True) rather than pr.remove_jobs_silently()..
  pr.remove_jobs_silently(recursive=True)


  0%|          | 0/3 [00:00<?, ?it/s]

## Dirichlet partition

In [3]:
job_d = pr.create.job.Fenics('fenics_drichlet')

/home/muhammad/workspace/gitRepo/pyiron-repos/pyiron_continuum/notebooks/pyiron_continuum/fenics/job/generic.py:138: UserWarning: Currently, the c++ dolfin functions used by fenics are not stored in the HDF5 file. This includes the domains, boundary condition, spatial functions. Therefore, it is not possible to reload the job properly, from HDF5 file. It would be safe to remove the Fenics jobs, after defining the project.
  warnings.warn("Currently, the c++ dolfin functions used by fenics are not stored in the HDF5 file."


### The domain and mesh

In [4]:
y_bottom, y_top = 0, 1
x_left, x_right = 0, 2
x_coupling = 1.0  # x coordinate of coupling interface
radius = 0.2
nx = ny = 9
job_d.domain = job_d.create.domain.regular_mesh.rectangle((x_left, y_bottom), 
                                                          (x_coupling, y_top), nx, ny, diagonal="left")

### Defining the coupled and non-coupled boundaries

#### Non-coupled boundary
Dirichlet boundary at the non-coupled boundaries, based on the expression `u_D`:

In [5]:
alpha = 3  # parameter alpha
beta = 1.3  # parameter beta
u_analytical = job_d.Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t', degree=2, alpha=alpha, beta=beta, t=0)
u_analytical_interpolated_dirichlet = job_d.interpolate_function(u_analytical)

In [6]:
conditions_non_coupling = 'not near(x[0], 1.0, 1E-14) \
                                    or near(x[1], 1.0, 1E-14) or near(x[1], 0., 1E-14)'   
non_coupling_bc_dirichlet = job_d.create.subdomain(conditions=conditions_non_coupling)
job_d.BCs.append(job_d.create.bc.dirichlet(expression=u_analytical,bc_fnc=non_coupling_bc_dirichlet))

#### Coupled boundary

In [7]:
conditions_coupling_bc = 'near(x[0], 1.0, 1E-14)'
coupling_bc_dirichlet = job_d.create.subdomain(conditions=conditions_coupling_bc)

### PDE equation

In [8]:
job_d.input.dt = 0.1

#### Initial Condition

In [9]:
job_d.u_n = u_analytical_interpolated_dirichlet
job_d.u_n.rename("Temperature", "")

In [10]:
f = job_d.Expression('beta - 2 - 2*alpha', degree=2, alpha=alpha, beta=beta)

In [11]:
job_d.F = job_d.u * job_d.v * job_d.dx + job_d.dt * job_d.dot(job_d.grad_u, job_d.grad_v) * job_d.dx \
        - (job_d.u_n + job_d.dt * f) * job_d.v * job_d.dx
job_d.solution.rename("Temperature", "")

### Precice Adaptor configuration

In [12]:
W_d = job_d.V_g.sub(0).collapse()
flux_normal = job_d.Expression("2 * x[0]", degree=1)
flux_normal_interpolated = job_d.interpolate_function(flux_normal, function_space=W_d)

In [13]:
job_d.adapter_conf = job_d.create.adapter_conf(config_file='precice_config/precice-adapter-config-D.json', coupling_boundary=coupling_bc_dirichlet,
                                                    write_object=flux_normal_interpolated)

In [14]:
def update_coupling_dirichlet(coupling_expression, coupling_boundary, job):
    job.BCs.append(job.create.bc.dirichlet(coupling_expression, coupling_boundary))

def coupling_data_dirichlet(config, job):
    job.flux.rename("Flux", "")
    job.cal_flux()
    flux_x = job.interpolate_function(job.flux.sub(0), function_space=config.write_object.function_space())
    return flux_x

In [15]:
job_d.adapter_conf.update_boundary_func = update_coupling_dirichlet
job_d.adapter_conf.coupling_data_func = coupling_data_dirichlet




## Neumann Partition

In [16]:
job_n = pr.create.job.Fenics('fenics_neumann')

In [17]:
job_n.domain = job_n.create.domain.regular_mesh.rectangle((x_coupling, y_bottom), 
                                                          (x_right, y_top), nx, ny, diagonal="left")

In [18]:
non_coupling_bc_neumann = job_n.create.subdomain(conditions=conditions_non_coupling)
job_n.BCs.append(job_n.create.bc.dirichlet(expression=u_analytical,bc_fnc=non_coupling_bc_neumann))

In [19]:
coupling_bc_neumann = job_n.create.subdomain(conditions=conditions_coupling_bc)

In [20]:
u_analytical_interpolated_neumann = job_n.interpolate_function(u_analytical)

In [21]:
job_n.input.dt = 0.1

In [22]:
job_n.u_n = u_analytical_interpolated_neumann
job_n.u_n.rename("Temperature", "")

In [23]:
f = job_n.Expression('beta - 2 - 2*alpha', degree=2, alpha=alpha, beta=beta, t=0)
job_n.F = job_n.u * job_n.v * job_n.dx + job_n.dt * job_n.dot(job_n.grad_u, job_n.grad_v) * job_n.dx \
        - (job_n.u_n + job_n.dt * f) * job_n.v * job_n.dx
job_n.solution.rename("Temperature", "")

In [24]:
job_n.adapter_conf = job_n.create.adapter_conf(config_file='precice_config/precice-adapter-config-N.json', coupling_boundary=coupling_bc_neumann,
                                               write_object=u_analytical_interpolated_neumann, function_space=job_n.V_g.sub(0).collapse())

In [25]:
def update_coupling_neumann(coupling_expression, coupling_boundary, job):
    job.F += job.v * coupling_expression * job.ds

def coupling_data_neumann(config, job):
    return job.u

In [26]:
job_n.adapter_conf.update_boundary_func = update_coupling_neumann
job_n.adapter_conf.coupling_data_func = coupling_data_neumann

### Creating Precice master job

In [27]:
job_p = pr.create.job.Precice('precice_parent')

**Appending fenics jobs as child job to the precice job**

In [28]:
job_p.child_list.append(job_d)
job_p.child_list.append(job_n)

**Running the precice job**

In [29]:
job_p.run()

The job precice_parent was saved and received the ID: 10
The job fenics_drichlet was saved and received the ID: 11
The job fenics_neumann was saved and received the ID: 12
---[precice]  This is preCICE version 2.3.0
---[precice]  Revision info: no-info [Git failed/Not a repository]
---[precice]  Configuration: Debug
---[precice]  Configuring preCICE with configuration "/home/muhammad/workspace/gitRepo/pyiron-repos/pyiron_continuum/notebooks/precice_config/precice-config.xml"
---[precice]  I am participant "Dirichlet"
---[precice]  Setting up master communication to coupling partner/s
---[precice]  This is preCICE version 2.3.0
---[precice]  Revision info: no-info [Git failed/Not a repository]
---[precice]  Configuration: Debug
---[precice]  Configuring preCICE with configuration "/home/muhammad/workspace/gitRepo/pyiron-repos/pyiron_continuum/notebooks/precice_config/precice-config.xml"
---[precice]  I am participant "Neumann"
---[precice]  Setting up master communication to coupling pa

In [30]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,10,finished,None,precice_parent,/precice_parent,/home/muhammad/,workspace/gitRepo/pyiron-repos/pyiron_continuum/notebooks/partitioned_heat_conduction/,2022-01-25 01:29:54.628646,NaT,NaN,pyiron@cmleo14#1,Precice,0.4,None,None
1,11,finished,None,fenics_drichlet,/fenics_drichlet,/home/muhammad/,workspace/gitRepo/pyiron-repos/pyiron_continuum/notebooks/partitioned_heat_conduction/,2022-01-25 01:29:54.707416,2022-01-25 01:29:56.480669,1.0,pyiron@cmleo14#1,Fenics,0.4,None,None
2,12,finished,None,fenics_neumann,/fenics_neumann,/home/muhammad/,workspace/gitRepo/pyiron-repos/pyiron_continuum/notebooks/partitioned_heat_conduction/,2022-01-25 01:29:54.715938,2022-01-25 01:29:56.514049,1.0,pyiron@cmleo14#1,Fenics,0.4,None,None
